<a href="https://colab.research.google.com/github/galitneu/auto-eda-tool/blob/main/eda_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>